<a href="https://colab.research.google.com/github/tthandi/magic_formula_project/blob/main/AdjustedDailyPriceLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
alpha_vantage_daily_price_auto.py  –  full-crawl OR gap-fill (auto)

Behavior
--------
* If the destination table has **no symbols**, run a **full crawl** (all AV symbols).
* If the table already has data, run a **gap-fill** for **missing** symbols only.
* Safe + fast: async fetch, rate-limited, WRITE_APPEND, per-partition flush locks.
* End-of-run de-dup cleans only the partitions we touched.

Auth
----
* Works in Colab (user auth), with a service-account JSON via
  GOOGLE_APPLICATION_CREDENTIALS, or with gcloud ADC.

Env
---
* Alpha Vantage key via ALPHAVANTAGE_API_KEY (falls back to literal if unset).
"""

# ───────────────────────────── CONFIG ───────────────────────────────
PROJECT   = ""      # GCP project ID
DATASET   = ""      # BigQuery dataset
TABLE     = ""      # Destination table name
CHECKFILE = "av_done_symbols.txt"           # success-only (not used to skip in full mode)
FAILFILE  = "av_failed_symbols.txt"         # failures recorded here

import os as _os
API_KEY   = _os.getenv("ALPHAVANTAGE_API_KEY") or ""

CALLS_PER_MIN  = 75                         # paid-tier limit (free=5)
CONCURRENCY    = 50                         # concurrent HTTP requests
FLUSH_ROWS     = 100_000                    # per-partition flush threshold
PART_START, PART_END = 199901, 204001       # RANGE partition yyyymm

# ───────────────────────────── IMPORTS ──────────────────────────────
import io, csv, time, asyncio, tempfile, aiohttp, nest_asyncio, sys
from collections import defaultdict
from typing import Dict, List, Set
import pandas as pd
import tqdm
from aiolimiter import AsyncLimiter

from google.cloud import bigquery
from google.api_core import exceptions as gex
from google.auth.exceptions import DefaultCredentialsError, RefreshError
from google.oauth2 import service_account

nest_asyncio.apply()

# ───────────────────────── Auth-robust BigQuery client ──────────────
def get_bq_client() -> bigquery.Client:
    # 1) Colab / Notebook user auth
    try:
        from google.colab import auth as colab_auth  # type: ignore
        colab_auth.authenticate_user()
        return bigquery.Client(project=PROJECT)
    except Exception:
        pass

    # 2) Service account via GOOGLE_APPLICATION_CREDENTIALS
    sa_path = _os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    if sa_path and _os.path.exists(sa_path):
        creds = service_account.Credentials.from_service_account_file(sa_path)
        return bigquery.Client(project=PROJECT, credentials=creds)

    # 3) Application Default Credentials (gcloud)
    #    Make sure you've run:  gcloud auth application-default login
    try:
        return bigquery.Client(project=PROJECT)
    except (DefaultCredentialsError, RefreshError) as e:
        raise RuntimeError(
            "No Google Cloud credentials found. Do one of:\n"
            "  • In Colab: from google.colab import auth; auth.authenticate_user()\n"
            "  • With service account: set GOOGLE_APPLICATION_CREDENTIALS to your JSON path\n"
            "  • With gcloud: run `gcloud auth application-default login`"
        ) from e

bq = get_bq_client()

# ───────────────────── Alpha Vantage async fetchers ─────────────────
BASE_URL     = "https://www.alphavantage.co/query"
RATE_LIMITER = AsyncLimiter(CALLS_PER_MIN, 60)

async def fetch_series(session: aiohttp.ClientSession, sym: str) -> pd.DataFrame | None:
    """Fetch TIME_SERIES_DAILY_ADJUSTED for a symbol; return tidy DataFrame or None."""
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": sym,
        "outputsize": "full",
        "apikey": API_KEY,
    }
    async with RATE_LIMITER:
        try:
            async with session.get(BASE_URL, params=params, timeout=45) as resp:
                if resp.status != 200:
                    return None
                data = await resp.json(content_type=None)
        except (aiohttp.ClientError, asyncio.TimeoutError):
            return None

    key = next((k for k in data if "Time Series" in k), None)
    if not key or not data.get(key):
        return None

    df = (
        pd.DataFrame.from_dict(data[key], orient="index")
          .reset_index()
          .rename(columns={
              "index": "date",
              "1. open": "open",
              "2. high": "high",
              "3. low": "low",
              "4. close": "close",
              "5. adjusted close": "adj_close",
              "6. volume": "volume",
              "7. dividend amount": "dividend",
              "8. split coefficient": "split_coef",
          })
    )
    df["symbol"] = sym
    df["date"]   = pd.to_datetime(df["date"]).dt.date
    df["yyyymm"] = df["date"].apply(lambda d: d.year * 100 + d.month).astype("int64")

    float_cols = ["open","high","low","close","adj_close","dividend","split_coef"]
    df[float_cols] = df[float_cols].apply(pd.to_numeric, errors="coerce")
    df["volume"]   = pd.to_numeric(df["volume"], errors="coerce").astype("Int64")
    return df

async def listing(session: aiohttp.ClientSession, state: str) -> List[str]:
    """Async, rate-limited Alpha Vantage LISTING_STATUS CSV → list of stock symbols."""
    params = {
        "function": "LISTING_STATUS",
        "state": state,           # "active" or "delisted"
        "datatype": "csv",
        "apikey": API_KEY,
    }
    async with RATE_LIMITER:
        try:
            async with session.get(BASE_URL, params=params, timeout=45) as resp:
                if resp.status != 200:
                    tqdm.tqdm.write(f"⚠️ listing {state} http {resp.status}")
                    return []
                txt = await resp.text()
        except (aiohttp.ClientError, asyncio.TimeoutError):
            tqdm.tqdm.write(f"⚠️ listing {state} fetch error")
            return []
    rdr = csv.DictReader(io.StringIO(txt))
    return [r["symbol"] for r in rdr if r.get("assetType") == "Stock" and r.get("symbol")]

# ───────────────────────── BigQuery bootstrapping ───────────────────
# Dataset
try:
    bq.get_dataset(f"{PROJECT}.{DATASET}")
except gex.NotFound:
    bq.create_dataset(bigquery.Dataset(f"{PROJECT}.{DATASET}"))

# Table (partitioned & clustered)
table_ref = f"{PROJECT}.{DATASET}.{TABLE}"
if not any(t.table_id == TABLE for t in bq.list_tables(f"{PROJECT}.{DATASET}")):
    schema = [
        bigquery.SchemaField("symbol", "STRING"),
        bigquery.SchemaField("date", "DATE"),
        bigquery.SchemaField("open", "FLOAT"),
        bigquery.SchemaField("high", "FLOAT"),
        bigquery.SchemaField("low", "FLOAT"),
        bigquery.SchemaField("close", "FLOAT"),
        bigquery.SchemaField("adj_close", "FLOAT"),
        bigquery.SchemaField("volume", "INT64"),
        bigquery.SchemaField("dividend", "FLOAT"),
        bigquery.SchemaField("split_coef", "FLOAT"),
        bigquery.SchemaField("yyyymm", "INT64"),
    ]
    tbl = bigquery.Table(table_ref, schema=schema)
    tbl.range_partitioning = bigquery.RangePartitioning(
        field="yyyymm",
        range_=bigquery.PartitionRange(PART_START, PART_END, 1),
    )
    tbl.clustering_fields = ["symbol"]
    bq.create_table(tbl)
    tqdm.tqdm.write(f"➡️  created table {table_ref}")

# Ensure we can read schema (for CSV order if needed)
tbl = bq.get_table(table_ref)

# ───────────────────────── Load to a single partition ───────────────
CSV_COLS = [
    "symbol", "date", "open", "high", "low", "close",
    "adj_close", "volume", "dividend", "split_coef", "yyyymm",
]

def load_partition(df_part: pd.DataFrame, yyyymm: int, retries: int = 3):
    """Append df_part to TABLE$yyyymm via gzip-CSV (no truncation)."""
    if df_part is None or df_part.empty:
        return
    df_part = df_part[CSV_COLS]
    fd, tmp = tempfile.mkstemp(suffix=".csv.gz"); _os.close(fd)
    df_part.to_csv(tmp, index=False, compression="gzip", date_format="%Y-%m-%d")

    dest = f"{table_ref}${yyyymm}"
    cfg = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=1,
        write_disposition="WRITE_APPEND",
    )
    for attempt in range(1, retries + 1):
        try:
            with open(tmp, "rb") as buf:
                job = bq.load_table_from_file(buf, dest, job_config=cfg, rewind=True)
            tqdm.tqdm.write(f"⬆️  load {dest} rows {len(df_part)} (try {attempt})")
            job.result(timeout=300)
            tqdm.tqdm.write(f"✅ partition {yyyymm} rows {len(df_part)}")
            break
        except (gex.GoogleAPICallError, gex.RetryError, TimeoutError) as e:
            tqdm.tqdm.write(f"⚠️  {dest} {e} (try {attempt})")
            time.sleep(10)
    _os.remove(tmp)

# ───────────────────────── Checkpoint & helpers ─────────────────────
def read_checkpoint() -> Set[str]:
    return set(open(CHECKFILE).read().splitlines()) if _os.path.exists(CHECKFILE) else set()

def add_checkpoint(sym: str):
    with open(CHECKFILE, "a") as f:
        f.write(sym + "\n")

def add_failure(sym: str):
    with open(FAILFILE, "a") as f:
        f.write(sym + "\n")

def symbols_already_loaded() -> Set[str]:
    """Return distinct symbols already in BigQuery (used for gap-fill)."""
    q = f"SELECT DISTINCT symbol FROM `{table_ref}`"
    return {row.symbol for row in bq.query(q).result()}

# ───────────────────────── De-duplication helper ────────────────────
def dedupe_partitions(parts: Set[int]):
    """De-duplicate rows within each provided yyyymm partition (one row per (symbol, date))."""
    import tqdm as _tqdm
    if not parts:
        _tqdm.tqdm.write("🧹 dedupe: no partitions to process")
        return

    sql = f"""
    -- De-duplicate within each requested partition (yyyymm).
    FOR part IN (SELECT p FROM UNNEST(@parts) AS p) DO
      CREATE OR REPLACE TEMP TABLE tmp AS
      SELECT symbol, date, open, high, low, close, adj_close, volume, dividend, split_coef, yyyymm
      FROM (
        SELECT t.*,
               ROW_NUMBER() OVER (PARTITION BY symbol, date ORDER BY yyyymm DESC) AS rn
        FROM `{table_ref}` AS t
        WHERE yyyymm = part
      )
      WHERE rn = 1;

      DELETE FROM `{table_ref}` WHERE yyyymm = part;

      INSERT INTO `{table_ref}` (
        symbol, date, open, high, low, close, adj_close, volume, dividend, split_coef, yyyymm
      )
      SELECT * FROM tmp;
    END FOR;
    """
    job = bq.query(
        sql,
        job_config=bigquery.QueryJobConfig(
            query_parameters=[bigquery.ArrayQueryParameter("parts", "INT64", sorted(parts))]
        ),
    )
    job.result()
    _tqdm.tqdm.write(f"🧹 dedupe: processed {len(parts)} partition(s)")

# ───────────────────────── Main async entrypoint ────────────────────
async def main():
    # Shared state
    sem                         = asyncio.Semaphore(CONCURRENCY)
    pending: Dict[int, List[pd.DataFrame]] = defaultdict(list)
    pending_rows: Dict[int, int]           = defaultdict(int)
    part_locks: Dict[int, asyncio.Lock]    = {}      # per-partition flush locks
    touched_parts: Set[int]                 = set()

    async def maybe_flush(part: int):
        """Flush a partition buffer if it exceeds FLUSH_ROWS (with per-partition lock)."""
        lock = part_locks.setdefault(part, asyncio.Lock())
        async with lock:
            if pending_rows[part] >= FLUSH_ROWS:
                batch = pd.concat(pending[part]); pending[part].clear(); pending_rows[part] = 0
                await asyncio.to_thread(load_partition, batch, part)

    # Single global session (listings + series)
    connector = aiohttp.TCPConnector(limit=CONCURRENCY)
    async with aiohttp.ClientSession(connector=connector, timeout=aiohttp.ClientTimeout(total=90)) as session:
        # Build AV universe
        active, delisted = await asyncio.gather(
            listing(session, "active"),
            listing(session, "delisted")
        )
        universe = sorted(set(active + delisted))

        # Decide mode
        existing = symbols_already_loaded()   # set of symbols in BQ
        if len(existing) == 0:
            mode = "full"
            # Full crawl: ignore local checkpoint when building todo to guarantee a truly full refill
            todo = universe
        else:
            mode = "gapfill"
            # Gap fill: rely on BQ to skip already-loaded symbols; ignore checkpoint here
            todo = [s for s in universe if s not in existing]

        tqdm.tqdm.write(
            f"Mode: {mode.upper()} | AV universe {len(universe):,} | "
            f"already in BQ {len(existing):,} | to fetch {len(todo):,}"
        )

        # Workers
        pbar = tqdm.tqdm(total=len(todo), desc="Symbols processed")
        async def worker(sym: str):
            async with sem:
                df = await fetch_series(session, sym)
                if df is not None and not df.empty:
                    for part, grp in df.groupby("yyyymm"):
                        pending[part].append(grp)
                        pending_rows[part] += len(grp)
                        touched_parts.add(int(part))
                        await maybe_flush(part)
                    # Record success; ignored for building gap-fill todo but useful for audits
                    add_checkpoint(sym)
                else:
                    add_failure(sym)
                pbar.update(1)

        tasks = [asyncio.create_task(worker(sym)) for sym in todo]
        await asyncio.gather(*tasks)
        pbar.close()

    # Final flush for any leftover partitions
    for part, dfs in pending.items():
        if dfs:
            await asyncio.to_thread(load_partition, pd.concat(dfs), part)

    # End-of-run de-dup only for touched partitions
    await asyncio.to_thread(dedupe_partitions, touched_parts)

# ────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    asyncio.run(main())
